# ***Step 1: Import libraries & Set up***

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
sequence_length = 28
input_size = 12 # N_mffc
hidden_size = 64
num_layers = 3
num_classes = 35
batch_size = 1
num_epochs = 25
learning_rate = 0.001

# ***Step 2: Use trained model (FP32)***
Reference: https://github.com/felixchenfy/Speech-Commands-Classification-by-LSTM-PyTorch

In [ ]:
# Recurrent neural network (many-to-one)
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        return x
model = LSTM(12, 35,64,3)
x = torch.randn(batch_size, sequence_length, input_size)
out = model(x)
print(out)



tensor([[[ 1.1618e+00, -1.2066e-01, -1.1227e+00, -8.7770e-01,  2.3996e+00,
           4.8252e-01, -1.9567e-01, -8.6777e-01,  1.0210e+00,  9.5329e-01,
           4.1807e-01, -6.4509e-01],
         [ 1.6898e-01, -1.5713e+00, -1.1249e+00,  1.8718e-02,  1.0463e+00,
          -1.5450e+00,  6.0701e-01,  4.4323e-01,  2.5037e+00,  1.6996e+00,
           9.1225e-01,  2.5026e-01],
         [ 1.0516e+00,  1.6390e+00,  3.8012e-01,  1.2767e-01,  7.6925e-01,
           7.1132e-01,  1.5308e+00, -5.2802e-01,  1.1973e+00,  1.0186e+00,
           1.7128e-01,  3.6479e-01],
         [ 2.0257e-02, -9.0348e-01, -4.6064e-01,  3.7012e-01, -1.0429e+00,
           2.8970e-01,  5.9412e-01,  1.5718e+00, -4.0005e-01, -1.1610e+00,
          -8.7107e-01, -1.8775e-01],
         [-1.5578e+00, -1.2727e-01, -7.2210e-02, -2.3299e+00,  2.6017e-01,
          -1.8751e-01, -2.1119e-01,  3.4054e-01,  5.0701e-01, -1.8462e-01,
          -2.1012e-01,  2.6056e-01],
         [ 1.0140e+00, -1.2731e+00, -1.3324e+00,  1.6690e+00,  3.

***--- Above LSTM model tensors representation for Affine per tensor quantization--***

In [ ]:
import os

In [ ]:
T = torch.tensor (out)
print(T)

tensor([[[ 1.1618e+00, -1.2066e-01, -1.1227e+00, -8.7770e-01,  2.3996e+00,
           4.8252e-01, -1.9567e-01, -8.6777e-01,  1.0210e+00,  9.5329e-01,
           4.1807e-01, -6.4509e-01],
         [ 1.6898e-01, -1.5713e+00, -1.1249e+00,  1.8718e-02,  1.0463e+00,
          -1.5450e+00,  6.0701e-01,  4.4323e-01,  2.5037e+00,  1.6996e+00,
           9.1225e-01,  2.5026e-01],
         [ 1.0516e+00,  1.6390e+00,  3.8012e-01,  1.2767e-01,  7.6925e-01,
           7.1132e-01,  1.5308e+00, -5.2802e-01,  1.1973e+00,  1.0186e+00,
           1.7128e-01,  3.6479e-01],
         [ 2.0257e-02, -9.0348e-01, -4.6064e-01,  3.7012e-01, -1.0429e+00,
           2.8970e-01,  5.9412e-01,  1.5718e+00, -4.0005e-01, -1.1610e+00,
          -8.7107e-01, -1.8775e-01],
         [-1.5578e+00, -1.2727e-01, -7.2210e-02, -2.3299e+00,  2.6017e-01,
          -1.8751e-01, -2.1119e-01,  3.4054e-01,  5.0701e-01, -1.8462e-01,
          -2.1012e-01,  2.6056e-01],
         [ 1.0140e+00, -1.2731e+00, -1.3324e+00,  1.6690e+00,  3.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


***--Maximum Value and minimum value of x***

In [ ]:
# Min-Max value of float_32 tensor (x) find out for scale (s) and zero point (z)

b = torch.max(T)
print(b)

tensor(2.8756)


In [ ]:
a = torch.min(T)
print(a)

tensor(-2.8897)


In [ ]:
# scale value

s = (b-a)/65535

print(s)

tensor(8.7972e-05)


In [ ]:
# zero point

z = torch.round(-a*65535/(b-a))

print(z)

tensor(32847.)


# ***Step-3: Apply Quantization by round and clipping function (Affine mapping)***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq = torch.clip(f, min=0, max=65535) # Here min & max value we can change as per Tbit.
# But, I have checked for 16 bit
print (Tq)
torch.save(Tq,'qtz_tensor.pt')


tensor([[[46054., 31475., 20085., 22870., 60124., 38332., 30623., 22983.,
          44453., 43683., 37599., 25514.],
         [34768., 14985., 20060., 33060., 44740., 15285., 39747., 37885.,
          61307., 52166., 43217., 35692.],
         [44801., 51478., 37168., 34298., 41591., 40933., 50248., 26845.,
          46457., 44426., 34794., 36994.],
         [33077., 22577., 27611., 37054., 20992., 36140., 39600., 50714.,
          28300., 19650., 22945., 30713.],
         [15139., 31400., 32026.,  6362., 35804., 30715., 30446., 36718.,
          38610., 30748., 30459., 35809.],
         [44374., 18376., 17701., 51818., 33254., 31021., 54840., 37724.,
          20753., 38729., 11214., 46515.],
         [36064., 34249., 53695., 23689., 34914., 36069., 38467., 19799.,
          18790., 42986., 47923., 14847.],
         [48270., 36622., 30757., 24082., 23800., 45963., 43851., 21294.,
          49259., 41168., 24603., 42355.],
         [44474., 39499., 48740., 18493., 21400., 31472., 26554.

# ***Step-4: Apply Dequantization***

In [ ]:
Tdq = s*(Tq - z)
print(Tdq)

tensor([[[ 1.1619e+00, -1.2070e-01, -1.1227e+00, -8.7770e-01,  2.3996e+00,
           4.8253e-01, -1.9565e-01, -8.6776e-01,  1.0210e+00,  9.5327e-01,
           4.1805e-01, -6.4510e-01],
         [ 1.6900e-01, -1.5714e+00, -1.1249e+00,  1.8738e-02,  1.0463e+00,
          -1.5450e+00,  6.0701e-01,  4.4321e-01,  2.5037e+00,  1.6995e+00,
           9.1227e-01,  2.5028e-01],
         [ 1.0516e+00,  1.6390e+00,  3.8013e-01,  1.2765e-01,  7.6923e-01,
           7.1135e-01,  1.5308e+00, -5.2801e-01,  1.1973e+00,  1.0186e+00,
           1.7128e-01,  3.6482e-01],
         [ 2.0234e-02, -9.0348e-01, -4.6062e-01,  3.7010e-01, -1.0429e+00,
           2.8969e-01,  5.9408e-01,  1.5718e+00, -4.0001e-01, -1.1610e+00,
          -8.7110e-01, -1.8773e-01],
         [-1.5578e+00, -1.2730e-01, -7.2225e-02, -2.3300e+00,  2.6013e-01,
          -1.8756e-01, -2.1122e-01,  3.4054e-01,  5.0699e-01, -1.8465e-01,
          -2.1008e-01,  2.6057e-01],
         [ 1.0141e+00, -1.2730e+00, -1.3324e+00,  1.6689e+00,  3.

# ***Step-5: MAE/MSE loss between T and Tdq***

***I. MAE loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 1.1618e+00, -1.2066e-01, -1.1227e+00, -8.7770e-01,  2.3996e+00,
           4.8252e-01, -1.9567e-01, -8.6777e-01,  1.0210e+00,  9.5329e-01,
           4.1807e-01, -6.4509e-01],
         [ 1.6898e-01, -1.5713e+00, -1.1249e+00,  1.8718e-02,  1.0463e+00,
          -1.5450e+00,  6.0701e-01,  4.4323e-01,  2.5037e+00,  1.6996e+00,
           9.1225e-01,  2.5026e-01],
         [ 1.0516e+00,  1.6390e+00,  3.8012e-01,  1.2767e-01,  7.6925e-01,
           7.1132e-01,  1.5308e+00, -5.2802e-01,  1.1973e+00,  1.0186e+00,
           1.7128e-01,  3.6479e-01],
         [ 2.0257e-02, -9.0348e-01, -4.6064e-01,  3.7012e-01, -1.0429e+00,
           2.8970e-01,  5.9412e-01,  1.5718e+00, -4.0005e-01, -1.1610e+00,
          -8.7107e-01, -1.8775e-01],
         [-1.5578e+00, -1.2727e-01, -7.2210e-02, -2.3299e+00,  2.6017e-01,
          -1.8751e-01, -2.1119e-01,  3.4054e-01,  5.0701e-01, -1.8462e-01,
          -2.1012e-01,  2.6056e-01],
         [ 1.0140e+00, -1.2731e+00, -1.3324e+00,  

***II. MSE Loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 1.1618e+00, -1.2066e-01, -1.1227e+00, -8.7770e-01,  2.3996e+00,
           4.8252e-01, -1.9567e-01, -8.6777e-01,  1.0210e+00,  9.5329e-01,
           4.1807e-01, -6.4509e-01],
         [ 1.6898e-01, -1.5713e+00, -1.1249e+00,  1.8718e-02,  1.0463e+00,
          -1.5450e+00,  6.0701e-01,  4.4323e-01,  2.5037e+00,  1.6996e+00,
           9.1225e-01,  2.5026e-01],
         [ 1.0516e+00,  1.6390e+00,  3.8012e-01,  1.2767e-01,  7.6925e-01,
           7.1132e-01,  1.5308e+00, -5.2802e-01,  1.1973e+00,  1.0186e+00,
           1.7128e-01,  3.6479e-01],
         [ 2.0257e-02, -9.0348e-01, -4.6064e-01,  3.7012e-01, -1.0429e+00,
           2.8970e-01,  5.9412e-01,  1.5718e+00, -4.0005e-01, -1.1610e+00,
          -8.7107e-01, -1.8775e-01],
         [-1.5578e+00, -1.2727e-01, -7.2210e-02, -2.3299e+00,  2.6017e-01,
          -1.8751e-01, -2.1119e-01,  3.4054e-01,  5.0701e-01, -1.8462e-01,
          -2.1012e-01,  2.6056e-01],
         [ 1.0140e+00, -1.2731e+00, -1.3324e+00,  

# ***Method-2:- Symmetric quantization scheme***

In [ ]:
f = torch.round(T/s + z)
print(f)
Tq1 = torch.clip(f, min=-32768, max=32768) # Here min & max value we can change as per Tbit.
# But, I have checked for 16 bit
print (Tq1)
torch.save(Tq1,'qtz_tensor.pt')


tensor([[[46054., 31475., 20085., 22870., 60124., 38332., 30623., 22983.,
          44453., 43683., 37599., 25514.],
         [34768., 14985., 20060., 33060., 44740., 15285., 39747., 37885.,
          61307., 52166., 43217., 35692.],
         [44801., 51478., 37168., 34298., 41591., 40933., 50248., 26845.,
          46457., 44426., 34794., 36994.],
         [33077., 22577., 27611., 37054., 20992., 36140., 39600., 50714.,
          28300., 19650., 22945., 30713.],
         [15139., 31400., 32026.,  6362., 35804., 30715., 30446., 36718.,
          38610., 30748., 30459., 35809.],
         [44374., 18376., 17701., 51818., 33254., 31021., 54840., 37724.,
          20753., 38729., 11214., 46515.],
         [36064., 34249., 53695., 23689., 34914., 36069., 38467., 19799.,
          18790., 42986., 47923., 14847.],
         [48270., 36622., 30757., 24082., 23800., 45963., 43851., 21294.,
          49259., 41168., 24603., 42355.],
         [44474., 39499., 48740., 18493., 21400., 31472., 26554.

In [ ]:
Tdq1 = s*(Tq1 - z)
print(Tdq1)

tensor([[[-0.0069, -0.1207, -1.1227, -0.8777, -0.0069, -0.0069, -0.1957,
          -0.8678, -0.0069, -0.0069, -0.0069, -0.6451],
         [-0.0069, -1.5714, -1.1249, -0.0069, -0.0069, -1.5450, -0.0069,
          -0.0069, -0.0069, -0.0069, -0.0069, -0.0069],
         [-0.0069, -0.0069, -0.0069, -0.0069, -0.0069, -0.0069, -0.0069,
          -0.5280, -0.0069, -0.0069, -0.0069, -0.0069],
         [-0.0069, -0.9035, -0.4606, -0.0069, -1.0429, -0.0069, -0.0069,
          -0.0069, -0.4000, -1.1610, -0.8711, -0.1877],
         [-1.5578, -0.1273, -0.0722, -2.3300, -0.0069, -0.1876, -0.2112,
          -0.0069, -0.0069, -0.1847, -0.2101, -0.0069],
         [-0.0069, -1.2730, -1.3324, -0.0069, -0.0069, -0.1606, -0.0069,
          -0.0069, -1.0639, -0.0069, -1.9031, -0.0069],
         [-0.0069, -0.0069, -0.0069, -0.8057, -0.0069, -0.0069, -0.0069,
          -1.1479, -1.2366, -0.0069, -0.0069, -1.5835],
         [-0.0069, -0.0069, -0.1839, -0.7711, -0.7959, -0.0069, -0.0069,
          -1.0163, -0.00

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(T, Tdq1)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[[ 1.1618e+00, -1.2066e-01, -1.1227e+00, -8.7770e-01,  2.3996e+00,
           4.8252e-01, -1.9567e-01, -8.6777e-01,  1.0210e+00,  9.5329e-01,
           4.1807e-01, -6.4509e-01],
         [ 1.6898e-01, -1.5713e+00, -1.1249e+00,  1.8718e-02,  1.0463e+00,
          -1.5450e+00,  6.0701e-01,  4.4323e-01,  2.5037e+00,  1.6996e+00,
           9.1225e-01,  2.5026e-01],
         [ 1.0516e+00,  1.6390e+00,  3.8012e-01,  1.2767e-01,  7.6925e-01,
           7.1132e-01,  1.5308e+00, -5.2802e-01,  1.1973e+00,  1.0186e+00,
           1.7128e-01,  3.6479e-01],
         [ 2.0257e-02, -9.0348e-01, -4.6064e-01,  3.7012e-01, -1.0429e+00,
           2.8970e-01,  5.9412e-01,  1.5718e+00, -4.0005e-01, -1.1610e+00,
          -8.7107e-01, -1.8775e-01],
         [-1.5578e+00, -1.2727e-01, -7.2210e-02, -2.3299e+00,  2.6017e-01,
          -1.8751e-01, -2.1119e-01,  3.4054e-01,  5.0701e-01, -1.8462e-01,
          -2.1012e-01,  2.6056e-01],
         [ 1.0140e+00, -1.2731e+00, -1.3324e+00,  

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", T)
print("Target Tensor:\n", Tdq1)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(T, Tdq1)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[[ 1.1618e+00, -1.2066e-01, -1.1227e+00, -8.7770e-01,  2.3996e+00,
           4.8252e-01, -1.9567e-01, -8.6777e-01,  1.0210e+00,  9.5329e-01,
           4.1807e-01, -6.4509e-01],
         [ 1.6898e-01, -1.5713e+00, -1.1249e+00,  1.8718e-02,  1.0463e+00,
          -1.5450e+00,  6.0701e-01,  4.4323e-01,  2.5037e+00,  1.6996e+00,
           9.1225e-01,  2.5026e-01],
         [ 1.0516e+00,  1.6390e+00,  3.8012e-01,  1.2767e-01,  7.6925e-01,
           7.1132e-01,  1.5308e+00, -5.2802e-01,  1.1973e+00,  1.0186e+00,
           1.7128e-01,  3.6479e-01],
         [ 2.0257e-02, -9.0348e-01, -4.6064e-01,  3.7012e-01, -1.0429e+00,
           2.8970e-01,  5.9412e-01,  1.5718e+00, -4.0005e-01, -1.1610e+00,
          -8.7107e-01, -1.8775e-01],
         [-1.5578e+00, -1.2727e-01, -7.2210e-02, -2.3299e+00,  2.6017e-01,
          -1.8751e-01, -2.1119e-01,  3.4054e-01,  5.0701e-01, -1.8462e-01,
          -2.1012e-01,  2.6056e-01],
         [ 1.0140e+00, -1.2731e+00, -1.3324e+00,  